In [7]:
%%capture
%run 04_account.ipynb
%run 05_block.ipynb

## Miner

Mining is a crucial component on any blockchain relying on proof of work (abbr. pow). It is used to reach consensus in a network of anonymous decentralized nodes. Anyone with an account `acc` can become a miner. 

The miner takes a new block and tries to solve a specific computational puzzle, a specific hash. This can only be done by brute force. 

`coinbase` creates a tx to the miner as a reward for providing the pow (also called coinbase tx). 

In [8]:
class Miner:
    def __init__(self, acc): 
        self.acc = acc
        self.pub = acc.pub
        
    def mine(self, txs, prev_header, diff, reward, attempts=1000):
        txs.insert(0, self.coinbase(reward))
        mt = MerkleTree(txs)
        bh = Header(mt.root, prev_header.hash, prev_header.number+1, len(txs))
        bh.diff   = diff
        bh.reward = reward
        bh.miner  = self.pub
        bh.mined  = True
        bh_b = bytes(bh)
        nonce = 0
        for i in range(attempts):
            candidate   = bh_b + str(nonce).encode()
            candidate_h = sha(candidate)
            if candidate_h[2:2+diff] == '0'*diff: break
            nonce += 1
            assert nonce != attempts, 'no nonce could be found'
        bh.nonce  = nonce
        return Block(bh, txs)
        
    def coinbase(self, reward): 
        return self.acc.sign(TX(self.pub, self.pub, reward, 0, self.acc.nonce))

### Genesis Block

Is the first block in the blockchain. This is how it all begins. 

In [9]:
def mine_genesis(txs):
    mt  = MerkleTree(txs)
    bh  = Header(mt.root, '0x0', 0, len(txs))
    return Block(bh, txs)

gb  = mine_genesis(r_stxs(2)); print(gb)

root:          💀 0x1aceb8e9b2...6d2
hash:          🕄 0xde2d3d5b47...da6
prev_hash:     0x0
number:        0
n_txs:         2
mined:         False
time:          1617485067.9508538 eth
volume:        10.0 eth
fees:          0.05 eth

txs:
📿 0x99fF -> 💝 0x37FF 8.0 eth
📿 0x99fF -> 💝 0x37FF 2.0 eth


### Mining

Mine a block on top of the genesis block

In [10]:
miner = Miner(acc1)
mb = miner.mine(r_stxs(10), gb.header, 1, 100); print(mb)

root:          📵 0x8fa74ce558...fac
hash:          💬 0x4626f8a850...103
prev_hash:     🕄 0xde2d3d5b47...da6
number:        1
n_txs:         11
mined:         True
time:          1617485069.1429892 eth
diff:          1
reward:        100
miner:         📿 0x99fF32a9F0...011
nonce:         5
volume:        166.0 eth
fees:          0.49 eth

txs:
📿 0x99fF -> 📿 0x99fF 100.0 eth
📿 0x99fF -> 💝 0x37FF 7.0 eth
📿 0x99fF -> 💝 0x37FF 9.0 eth
📿 0x99fF -> 💝 0x37FF 5.0 eth
📿 0x99fF -> 💝 0x37FF 7.0 eth
📿 0x99fF -> 💝 0x37FF 8.0 eth
📿 0x99fF -> 💝 0x37FF 9.0 eth
📿 0x99fF -> 💝 0x37FF 8.0 eth
📿 0x99fF -> 💝 0x37FF 6.0 eth
📿 0x99fF -> 💝 0x37FF 2.0 eth
📿 0x99fF -> 💝 0x37FF 5.0 eth


### PoW validation

We can validate a mined blocks pow very easily.

In [5]:
def val_pow(mb):
    bh          = mb.header
    mb_b        = bytes(bh)
    candidate   = mb_b + str(bh.nonce).encode()
    candidate_h = sha(candidate)
    if candidate_h[2:2+bh.diff] == '0'*bh.diff: return True
    else                                      : return False
    
assert val_pow(mb)

If we change something in the block the pow should break.

In [6]:
mb.header.prev_hash = rh()
assert not val_pow(mb)